In [1]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from hello.hello import HelloApp, TrueError
from time import time

In [3]:
ip = '192.168.1.86'

In [4]:
root = tk.Tk()
p = RTPlotWidget(root, max_pts=3600)
p.tkcanvas.config(height=400, width=600)

In [5]:
h = HelloApp(ip)
h.login()
h.getMainValues()
pass

In [6]:
h.getAdvancedValues()

{'CO2Duty(%)': 7.0,
 'CondenserDuty': 50.0,
 'FilterOvenDutyUser(%)': 8.0,
 'FilterOvenSP(C)': 35.0,
 'GasOut%': 35.90206909179687,
 'GasOutLPM': 0.49915909767151,
 'Lq Duty (%)': 0.0,
 'MFCAirFlowFeedback(LPM)': 0.31226140260696,
 'MFCCO2FlowFeedback(LPM)': 0.0349324345588684,
 'MFCN2FlowFeedback(LPM)': 0.0,
 'MFCO2FlowFeedback(LPM)': 0.15196524560452,
 'MainHeatDuty(%)': 3.96758484840393,
 'ManGasOut%M A': 0.5,
 'N2 Flow (%)': 0.0,
 'O2 Flow (%)': 30.24806594848633,
 'ReqMainGasMode A': 1,
 'SH Duty': 0.0,
 'SH Setpoint A': 0.0,
 'User SH Duty A': 0.0}

In [7]:
def actual_rate(n2, o2, co2):
    _, pc_1, _ = pc(0, o2, co2)
    _, pc_2, _ = pc(n2, o2, co2)
    ratio = pc_1/pc_2
    actual_n2 = n2 * ratio
    return actual_n2

def pc(a, b, c):
    t = a + b + c
    return a/t, b/t, c/t

In [8]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [9]:
def tk_task(tk, interval, f):
    def fn(*args, **kw):
        try:
            return f(*args, **kw)
        finally:
            tk.after(interval, fn)
    tk.after(interval, fn)

In [10]:
def update():
    mv = h.getMainValues()
    do = mv['do']
    pv = do['pv']
    t = get_time()
    p.add_xy(t, pv)
    update_labels(mv)

In [11]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [12]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [13]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
dos = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    dos[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
mk_label("O2 Flow %", 0, 2)
mk_label("O2 Flow LPM", 0, 3)
    
def update_labels(mv):
    while True:
        try:
            av = h.getAdvancedValues()
        except TrueError:
            pass
        else:
            break

    dos['PV'].set_value(mv['do']['pv'])
    dos['O2 Flow %'].set_value(av['O2 Flow (%)'])
    dos['O2 Flow LPM'].set_value(av['MFCO2FlowFeedback(LPM)'])
    

In [ ]:
p.grid(rowspan=20, column=0, row=0)    
rbtn = tk.Button(root, text="Reset", command=lambda: p.reset())
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
tk_task(root, 500, update)
root.mainloop()

In [ ]:
p.tkcanvas

In [ ]:
btn._w